In [22]:
import pandas as pd
data = pd.read_csv("clean-features.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306304 entries, 0 to 306303
Data columns (total 8 columns):
hair          306304 non-null object
eyes          306304 non-null object
age           306304 non-null object
trueAnswer    306304 non-null object
faceId        306304 non-null object
question      304928 non-null object
ethnicity     297673 non-null object
gender        306304 non-null object
dtypes: object(8)
memory usage: 18.7+ MB


In [23]:
data = data.dropna(subset=['question'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304928 entries, 0 to 306303
Data columns (total 8 columns):
hair          304928 non-null object
eyes          304928 non-null object
age           304928 non-null object
trueAnswer    304928 non-null object
faceId        304928 non-null object
question      304928 non-null object
ethnicity     296343 non-null object
gender        304928 non-null object
dtypes: object(8)
memory usage: 20.9+ MB


In [6]:
#load the pre-trained model, NLPL - English Wikipedia Dump of February 2017 - 3

In [4]:
from gensim.models import KeyedVectors
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# Access vectors for specific words with a keyed lookup:
model['man']

array([ 0.32617188,  0.13085938,  0.03466797, -0.08300781,  0.08984375,
       -0.04125977, -0.19824219,  0.00689697,  0.14355469,  0.0019455 ,
        0.02880859, -0.25      , -0.08398438, -0.15136719, -0.10205078,
        0.04077148, -0.09765625,  0.05932617,  0.02978516, -0.10058594,
       -0.13085938,  0.001297  ,  0.02612305, -0.27148438,  0.06396484,
       -0.19140625, -0.078125  ,  0.25976562,  0.375     , -0.04541016,
        0.16210938,  0.13671875, -0.06396484, -0.02062988, -0.09667969,
        0.25390625,  0.24804688, -0.12695312,  0.07177734,  0.3203125 ,
        0.03149414, -0.03857422,  0.21191406, -0.00811768,  0.22265625,
       -0.13476562, -0.07617188,  0.01049805, -0.05175781,  0.03808594,
       -0.13378906,  0.125     ,  0.0559082 , -0.18261719,  0.08154297,
       -0.08447266, -0.07763672, -0.04345703,  0.08105469, -0.01092529,
        0.17480469,  0.30664062, -0.04321289, -0.01416016,  0.09082031,
       -0.00927734, -0.03442383, -0.11523438,  0.12451172, -0.02

In [31]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

In [29]:
word = model['boy']
print(word[0:5])
print(word.shape, word.dtype)

[ 0.23535156  0.16503906  0.09326172 -0.12890625  0.01599121]
(300,) float32


In [5]:
# this function looks up the word vectors of the words in the questions, 
# adds the arrays together 
# and returns every feature as part of the array
# (since this function takes very long counter was added)

import numpy as np
counter = 0
all_rows = []
seen_vectors_by_question = {}
print('starting')
for index, item in data.iterrows():
    counter += 1
    if counter % 10000 == 0 or counter == 0:
        print('Processing %s. item' % counter)

    row      = item.get_values()
    question = row[5]

    vectors = []
    if question in seen_vectors_by_question:
        vectors = seen_vectors_by_question[question]
        if len(vectors) > 0:
            vectors = seen_vectors_by_question[question]
    else:
        for word in question.split():        
            if word in model:
                if len(vectors) == 0:
                    vectors = np.array(model[word])
                else:
                    vectors += np.array(model[word])
    
    seen_vectors_by_question[question] = vectors
    
    if len(vectors) == 0:
        continue

    row = np.append(row, vectors)
    all_rows.append(row)

print('Done. All count %s' % len(all_rows))
# all_rows

starting
Processing 10000. item
Processing 20000. item
Processing 30000. item
Processing 40000. item
Processing 50000. item
Processing 60000. item
Processing 70000. item
Processing 80000. item
Processing 90000. item
Processing 100000. item
Processing 110000. item
Processing 120000. item
Processing 130000. item
Processing 140000. item
Processing 150000. item
Processing 160000. item
Processing 170000. item
Processing 180000. item
Processing 190000. item
Processing 200000. item
Processing 210000. item
Processing 220000. item
Processing 230000. item
Processing 240000. item
Processing 250000. item
Processing 260000. item
Processing 270000. item
Processing 280000. item
Processing 290000. item
Processing 300000. item
Done. All count 304480


In [6]:
#writing all_rows to csv
columns  = ['hair', 'eyes', 'age','true_answer', 'FaceID', 'question', 'ethnicity', 'gender']

vwList = []
for i in range(1, 301): 
    vwList.append("wv"+str(i))
columns.extend(vwList)

In [7]:
import csv
csv_file = open("data/word2vec-data.csv", "w")

# columns = ['faceId', 'age', 'ethnicity', 'gender', 'hair', 'questions', 'responses']
csv_writer = csv.writer(csv_file)
# csv_writer.writeheader()
csv_writer.writerow(columns)
csv_writer.writerows(all_rows)

csv_file.close()

In [10]:
import pandas as pd
data = pd.read_csv("data/word2vec-data.csv")
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304480 entries, 0 to 304479
Columns: 308 entries, hair to wv300
dtypes: float64(300), object(8)
memory usage: 715.5+ MB


,hair,eyes,age,true_answer,FaceID,question,ethnicity,gender,wv1,wv2,...,wv291,wv292,wv293,wv294,wv295,wv296,wv297,wv298,wv299,wv300
0,blond,brown,25-34,yes,IMG_9891,hair long,white,female,-0.290039,0.291016,...,-0.130371,0.390625,-0.30957,0.160645,-0.047852,0.0,0.195801,0.494141,-0.107445,0.102051
1,brown,brown,25-34,yes,IMG_0153,hair long,white,female,-0.290039,0.291016,...,-0.130371,0.390625,-0.30957,0.160645,-0.047852,0.0,0.195801,0.494141,-0.107445,0.102051
2,brown,brown,45-54,no,IMG_0446,hair long,white,female,-0.290039,0.291016,...,-0.130371,0.390625,-0.30957,0.160645,-0.047852,0.0,0.195801,0.494141,-0.107445,0.102051
3,blond,blue,35-44,no,IMG_9484,hair long,white,female,-0.290039,0.291016,...,-0.130371,0.390625,-0.30957,0.160645,-0.047852,0.0,0.195801,0.494141,-0.107445,0.102051
4,brown,brown,25-34,yes,IMG_9846,hair long,white,male,-0.290039,0.291016,...,-0.130371,0.390625,-0.30957,0.160645,-0.047852,0.0,0.195801,0.494141,-0.107445,0.102051


In [19]:
model['boy']

array([ 2.35351562e-01,  1.65039062e-01,  9.32617188e-02, -1.28906250e-01,
        1.59912109e-02,  3.61328125e-02, -1.16699219e-01, -7.32421875e-02,
        1.38671875e-01,  1.15356445e-02,  1.87500000e-01, -2.91015625e-01,
        1.70898438e-02, -1.84570312e-01, -2.87109375e-01,  2.54821777e-03,
       -2.19726562e-01,  1.77734375e-01, -1.20605469e-01,  5.39550781e-02,
        3.78417969e-02,  2.49023438e-01,  1.76757812e-01,  2.69775391e-02,
        1.21093750e-01, -3.51562500e-01, -5.83496094e-02,  1.22070312e-01,
        5.97656250e-01, -1.60156250e-01,  1.08398438e-01, -2.40478516e-02,
       -1.16699219e-01,  3.58886719e-02, -2.37304688e-01,  1.15234375e-01,
        5.27343750e-01, -2.18750000e-01, -4.54101562e-02,  3.30078125e-01,
        3.75976562e-02, -5.51757812e-02,  3.26171875e-01,  6.74438477e-03,
        3.71093750e-01,  3.68652344e-02,  6.68945312e-02,  5.17578125e-02,
       -4.76074219e-02, -7.91015625e-02,  4.46777344e-02,  1.67968750e-01,
        5.51757812e-02, -

In [ ]:
#load the data again
import pandas as pd
data = pd.read_csv("clean-features.csv")
data = data.dropna(subset=['question'])

In [24]:
# now we will do the same function with averaging the words in the questions
import numpy as np
counter = 0
all_rows = []
seen_vectors_by_question = {}
print('starting')
for index, item in data.iterrows():
    counter += 1
    if counter % 10000 == 0 or counter == 0:
        print('Processing %s. item' % counter)

    row      = item.get_values()
    question = row[5]
    words    = question.split()

    vectors = []
    if question in seen_vectors_by_question:
        vectors_seen_before = seen_vectors_by_question[question]
        if len(vectors_seen_before) > 0:
            vectors = vectors_seen_before
    else:
        for word in words:        
            if word in model:
                if len(vectors) == 0:
                    vectors = np.array(model[word])
                else:
                    vectors += np.array(model[word])
        seen_vectors_by_question[question] = vectors

    if len(vectors) == 0:
        # when we can't find vectors for the question
        # we skip adding it to all_rows. is that what we want?
        continue

    vectors_avg = [x / len(words) for x in vectors]

    row = np.append(row, vectors_avg)
    all_rows.append(row)

print('Done. All count %s' % len(all_rows))


starting
Processing 10000. item
Processing 20000. item
Processing 30000. item
Processing 40000. item
Processing 50000. item
Processing 60000. item
Processing 70000. item
Processing 80000. item
Processing 90000. item
Processing 100000. item
Processing 110000. item
Processing 120000. item
Processing 130000. item
Processing 140000. item
Processing 150000. item
Processing 160000. item
Processing 170000. item
Processing 180000. item
Processing 190000. item
Processing 200000. item
Processing 210000. item
Processing 220000. item
Processing 230000. item
Processing 240000. item
Processing 250000. item
Processing 260000. item
Processing 270000. item
Processing 280000. item
Processing 290000. item
Processing 300000. item
Done. All count 304480


In [25]:
import csv

csv_file = open("data/word2vec-avg-data.csv", "w")

# columns = ['faceId', 'age', 'ethnicity', 'gender', 'hair', 'questions', 'responses']
csv_writer = csv.writer(csv_file)
# csv_writer.writeheader()
csv_writer.writerow(columns)
csv_writer.writerows(all_rows)

csv_file.close()

In [26]:
data = pd.read_csv("data/word2vec-avg-data.csv")
data.head()

,hair,eyes,age,true_answer,FaceID,question,ethnicity,gender,wv1,wv2,...,wv291,wv292,wv293,wv294,wv295,wv296,wv297,wv298,wv299,wv300
0,blond,brown,25-34,yes,IMG_9891,hair long,white,female,-0.14502,0.145508,...,-0.065186,0.195312,-0.154785,0.080322,-0.023926,0.0,0.0979,0.24707,-0.053722,0.051025
1,brown,brown,25-34,yes,IMG_0153,hair long,white,female,-0.14502,0.145508,...,-0.065186,0.195312,-0.154785,0.080322,-0.023926,0.0,0.0979,0.24707,-0.053722,0.051025
2,brown,brown,45-54,no,IMG_0446,hair long,white,female,-0.14502,0.145508,...,-0.065186,0.195312,-0.154785,0.080322,-0.023926,0.0,0.0979,0.24707,-0.053722,0.051025
3,blond,blue,35-44,no,IMG_9484,hair long,white,female,-0.14502,0.145508,...,-0.065186,0.195312,-0.154785,0.080322,-0.023926,0.0,0.0979,0.24707,-0.053722,0.051025
4,brown,brown,25-34,yes,IMG_9846,hair long,white,male,-0.14502,0.145508,...,-0.065186,0.195312,-0.154785,0.080322,-0.023926,0.0,0.0979,0.24707,-0.053722,0.051025


In [43]:
pd.set_option('display.max_colwidth', 40)
data.sample(n = 5)

,hair,eyes,age,true_answer,FaceID,question,ethnicity,gender,wv1,wv2,...,wv291,wv292,wv293,wv294,wv295,wv296,wv297,wv298,wv299,wv300
199937,black,brown,15-24,no,IMG_8622,boy,african,male,0.235352,0.165039,...,0.019043,-0.010315,-0.115234,-0.291016,-0.059570,-0.053711,-0.074219,-0.265625,-0.103027,0.135742
52702,black,dark brown,25-34,no,IMG_9469,wear glasses,african,female,-0.215332,-0.088928,...,0.188568,0.051758,-0.032974,0.086182,-0.024658,-0.170708,0.221680,-0.111572,0.234863,0.118988
126445,black,dark brown,15-24,no,IMG_8243,have ponytail,white,female,-0.070034,-0.009277,...,-0.033264,0.132812,0.104736,0.026855,0.148682,-0.080444,0.163330,-0.005859,-0.025398,0.142578
177155,brown,dark brown,15-24,no,IMG_0746,scarf have flowers,african,female,-0.026001,0.043294,...,-0.128662,-0.023234,-0.000977,0.108073,0.060384,0.040609,0.164144,-0.025391,0.095535,0.192383
189931,black,brown,15-24,yes,IMG_9574,hair bun,african,female,-0.241211,0.103882,...,-0.011230,0.169434,-0.239258,0.324219,0.253418,-0.111267,0.068604,0.160278,-0.137939,0.233398
